1. Introducción y Objetivo
El principal objetivo del actual proyecto es poder predecir el **precio de las viviendas en Ames**, basandose en las caracteristicas que el dataset ofrece.

Entre el objetivo:
- Seleccionar las variables principales.
- Entrenar un modelo de regresión.
- Medir el desempeño del modelo.
- Realizar un analisis acerca de cuales fueron las variables que mayor impacto tuvieon en el precio final de la vivienda.

2. Importar Librerías


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

3. Carga de Datos


In [ ]:
url = 'https://raw.githubusercontent.com/Aaronruizdiaz/ProyectoDSParteI-RuizDiaz.ipynb/refs/heads/main/AmesHousing.csv'

df = pd.read_csv(url)

df.shape
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

4. Hipótesis

El tamaño de la vivienda (superficie total) y la calidad de construcción
son los factores que más influyen en el precio de venta (SalePrice).

La hipótesis planteada es que el tamaño de la vivienda, es decir la superficie total y la calidad de la construcción son los factores mas relevanes en el precio e venta o **SalePrice**

5. Análisis Exploratorio de Datos (EDA)


5.1. Revisar valores faltantes

In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
# Como mi columnas no son todas numericas no puedo reemplazar todos los Nulos por una variable numérica como lo es la mediana.
# Es por esto que para las columnas numéricas utilizaremos la mediana mientras que para las columnas que no son numericas usaremos la moda.

# Columnas numéricas - acá usamos la mediana
num_columnas= df.select_dtypes(include=np.number).columns # para extraer solo las columnas con datos numéricos.
df[num_columnas] = df[num_columnas].fillna(df[num_columnas].median()) # rellenamos los NaN con la mediana

# Columnas categóricas - acá el valor que mas se repite, la moda
df[cat_columnas] = df[cat_columnas].fillna(df[cat_columnas].mode())

In [ ]:
# Como mi columnas no son todas numericas no puedo reemplazar todos los Nulos por una variable numérica como lo es la mediana.
# Es por esto que para las columnas numéricas utilizaremos la mediana mientras que para las columnas que no son numericas usaremos la moda.

# Columnas numéricas - acá usamos la mediana
num_columnas = df.select_dtypes(include=np.number).columns
df[num_columnas] = df[num_columnas].fillna(df[num_columnas].median())

# Columnas categóricas - acá el valor que mas se repite, la moda
cat_columnas = df.select_dtypes(exclude=np.number).columns
for col in cat_columnas:
    df[col] = df[col].fillna(df[col].mode()[0])

5.2. Distribución de la variable objetivo ('target')

In [ ]:
# En este bloque vamos analizar nuestra variable Target.
# Para esto tenemos un histograma que grafica la distribución de precios.

sns.histplot(df['SalePrice'])
plt.title("Distribución de SalePrice")
plt.show()

5.3. Análisis de variables numéricas

In [ ]:
numerical_features = df.select_dtypes(include=np.number).columns.tolist()
numerical_features

In [ ]:
# Vamos a listar todas las columnas codificadas numericamente
numerical_features = df.select_dtypes(include=np.number).columns.tolist()

# Utilizando solamente las numéricas continuas realizamos histogramas.
df[numerical_continuous].hist(figsize=(12, 8), bins=20)
plt.suptitle('Histogramas de Variables Numéricas Continuas', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Vamos a chequear la relación entre las variables numericas y nuestra variable TArget.
print("\nScatterplots de Variables Numéricas Continuas vs SalePrice:")

for col in numerical_continuous:
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=col, y='SalePrice', data=df, alpha=0.5) # Elegimos un scatterplot para detectar tendencias.
    plt.title(f'{col} vs SalePrice')
    plt.xlabel(col)
    plt.ylabel('SalePrice')
    plt.show()


5.4. Análisis de variables categóricas

In [ ]:
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_coded_num = ['OverallQual', 'OverallCond', 'MoSold', 'YrSold'] # Incluimos las que son codificadas numericamente
categorical_features = categorical_coded_num + df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='Overall Qual', data=df, hue='Sale Condition')
plt.title('Cantidad de casas por OverallQual y SaleCondition')
plt.xlabel('OverallQual (Calidad de la casa)')
plt.ylabel('Cantidad de casas')
plt.legend(title='Grupo de venta')
plt.show()

6. Feature Engineering (Ingeniería de Características)

In [ ]:
# Crear nuevas caracteristicas para nuestro Dataframe, en este caso la antiguedad de la casa
bins = [0, 1950, 1970, 1990, 2010, df['Year Built'].max() +1]
labels = ['Muy Vieja', 'Vieja', 'Media', 'Reciente', 'Muy Reciente']
df['AgeGroup'] = pd.cut(df['Year Built'], bins=bins, labels=labels, right=False)

In [ ]:
df[['Year Built', 'AgeGroup']].head()

In [ ]:
# Realizamos un gráfico tipo Countplot donde comparamos nuestra nueva característica

plt.figure(figsize=(10, 6))
sns.countplot(x='AgeGroup', data=df, hue='Sale Condition')
plt.title('Distribución de Grupos de Antigüedad por Grupo')
plt.xlabel('Grupo de Antigüedad de la Casa')
plt.ylabel('Cantidad de Casas')
plt.legend(title='Grupo de venta', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Lista completa de características a usar (sin 'SalePrice')
# Numéricas:
Features = df.select_dtypes(include=np.number).columns.tolist()
Target = 'SalePrice'

# Variables categóricas + nuestra nueva variable:
categorical_features_for_encoding = categorical_features + ['AgeGroup']

print(f"\nCaracterísticas numéricas a escalar: {numerical_features_for_scaling}")
print(f"Características categóricas a codificar (OneHot): {categorical_features_for_encoding}")
print(f"Variable objetivo: {target}")

7. Preprocesamiento de Datos

In [ ]:
# Definimos las características x y variable objetivo y
X = df[Features]
y = df['SalePrice']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_for_scaling),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_for_encoding)
    ],
    remainder='passthrough'
)

In [ ]:
# Creamos el procesador

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_for_scaling),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_for_encoding)
    ],
    remainder='passthrough'  # Si hubiera columnas no especificadas, las dejaría pasar
)

In [ ]:
preprocessor

8. División de Datos (Train/Test Split)

In [ ]:
# Dividir los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f"Tamaño del conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Tamaño del conjunto de prueba (X_test): {X_test.shape}")
print(f"Tamaño del conjunto de entrenamiento (y_train): {y_train.shape}")
print(f"Tamaño del conjunto de prueba (y_test): {y_test.shape}")

In [ ]:
print("\nDistribución de la variable objetivo en Train vs Test:")
print(f"Train: \n{y_train.value_counts(normalize=True)}")
print(f"Test: \n{y_test.value_counts(normalize=True)}")

9. Construcción y Entrenamiento del Modelo (Random Forest)

In [ ]:
# En este bloque tuvimos que redefinir las variables numéricas y las categóricas
# Porque encontramos errores en los nombres que generaban problemas.


target = 'SalePrice'

# Columnas numéricas y categóricas
numerical_features_for_scaling = df.select_dtypes(include=np.number).columns.tolist()
if target in numerical_features_for_scaling:
    numerical_features_for_scaling.remove(target)

categorical_features_for_encoding = df.select_dtypes(include=['object']).columns.tolist()

# Columnas numéricas como categóricas
categorical_coded_num = ['Overall Qual', 'Overall Cond', 'Mo Sold', 'Yr Sold']

# Sacamos las que estaban duplicadas
numerical_features_for_scaling = [col for col in numerical_features_for_scaling if col not in categorical_coded_num]
categorical_features_for_encoding += categorical_coded_num
categorical_features_for_encoding = list(set(categorical_features_for_encoding))

# Filtramos
numerical_features_for_scaling = [col for col in numerical_features_for_scaling if col in df.columns]
categorical_features_for_encoding = [col for col in categorical_features_for_encoding if col in df.columns]

# Ahora si trabajamos con el procesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_for_scaling),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_for_encoding)
    ],
    remainder='passthrough'
)

# Re Dividimos los daros
X = df[numerical_features_for_scaling + categorical_features_for_encoding]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Pipeline

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# -------------------------------
# 7. Entrenar el Pipeline completo
model_pipeline.fit(X_train, y_train)

print("Modelo entrenado correctamente")


In [ ]:
# Entrenar el pipeline completo (preprocesamiento + modelo)
model_pipeline.fit(X_train, y_train)

10. Evaluación del Modelo

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

In [ ]:
# Utilizando metricas de regresión vamos a evaluar el modelo

mae = mean_absolute_error(y_test, y_pred) # Métricas
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Evaluación del Modelo Regresión")
print(f"MAE  : {mae:.2f}")
print(f"MSE  : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.4f}")


10.1 Errores de predicción

In [ ]:
# Permite evaluar errores, lo que no esta cerca de la linea roja estaría mal

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Valores Reales vs Predichos')
plt.show()

12. Conclusiones

In [ ]:
print("\nValidación de Hipótesis:")
print(f"- Hipótesis: Las características 'GrLivArea', 'Overall Qual' y 'Neighborhood' influyen significativamente en el precio de las casas.")
print(f"- Las 10 variables más importantes incluyen: {Features}")
print(f"- R² del modelo en conjunto de prueba: {r2:.4f}")
print(f"- MAE del modelo: {mae:.2f}")
print("- Hipótesis: Las características 'GrLivArea', 'Overall Qual' y 'Neighborhood'son las mas influyentes en el precio de las casas.")
print("- Luego de ver las 10 variables mayores podes determinar si tienen relevancia.")
print("- Los gráficos des permiten saber si el modelo esta correcto")